# 🧪 Finance ABAC Demo - Step 4: Test Functions

## 📋 Overview
This notebook **tests all functions** created in the Finance ABAC demo.

### What This Notebook Does:
1. **Verifies Data**: Confirms all tables have correct row counts
2. **Tests Each Function**: Runs before/after examples for every masking function
3. **Validates Output**: Ensures masked data meets requirements
4. **Demonstrates Usage**: Shows how to apply functions in real queries

### Why Test Functions?
Testing ensures:
- **Correctness**: Functions work as designed
- **Data Integrity**: Original data isn't corrupted
- **Performance**: Functions execute efficiently
- **Compliance**: Masking meets regulatory requirements
- **User Experience**: Masked output is appropriate for different roles

### What You'll See:
For each function, you'll see:
- **Original Data**: Unmasked values from tables
- **Masked Data**: Transformed values after function application
- **Side-by-Side Comparison**: Before and after for easy validation

## 🎓 How to Use This Notebook
1. **Ensure Steps 1-3 Complete**: All functions, tables, and data must exist
2. **Run All Cells**: Execute sequentially to see all test results
3. **Review Output**: Compare original vs masked data
4. **Verify Expectations**: Check that masking behavior is appropriate

## ⚙️ Prerequisites
- ✅ **Step 1 completed**: All functions created
- ✅ **Step 2 completed**: Core schema with data
- ✅ **Step 3 completed**: Extended tables with data
- ✅ SELECT permission on all tables and functions

## 📊 Expected Results
After running this notebook:
- ✅ Table row counts displayed
- ✅ Each masking function tested with real data
- ✅ Before/after comparisons shown
- ✅ Confidence that ABAC setup is working correctly

## 🎯 What Comes Next?
After validating functions:
1. **Create Groups/Users**: Set up roles for ABAC policies
2. **Apply Tags**: Tag columns with sensitivity classifications
3. **Create Policies**: Build ABAC policies using these masking functions
4. **Test Access Control**: Verify different users see different data

---


In [0]:
%python
# 📋 Load Configuration from config.yaml
import yaml
from pathlib import Path

config_file = Path('config.yaml')
if config_file.exists():
    with open(config_file) as f:
        config = yaml.safe_load(f)
    CATALOG = config['catalog']
    SCHEMA = config['schema']
    print(f'✅ Configuration loaded from config.yaml')
    print(f'   📊 Catalog: {CATALOG}')
    print(f'   📁 Schema: {SCHEMA}')
else:
    # Fallback defaults
    CATALOG = 'your_catalog_name'
    SCHEMA = 'finance'
    print(f'⚠️  config.yaml not found - using defaults')
    print(f'   📊 Catalog: {CATALOG}')
    print(f'   📁 Schema: {SCHEMA}')

# Set catalog and schema to use in following cells
spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"USE SCHEMA {SCHEMA}")


## ⚙️ Configuration

Testing functions in:
- **Catalog**: `your_catalog_name`
- **Schema**: `finance`


In [0]:
SELECT '🧪 Testing functions in: ' || current_catalog() || '.' || current_schema() AS status;

In [0]:
-- =============================================
-- FINANCE DATA VERIFICATION & TESTING
-- =============================================
-- Purpose: Verify tables, data, and demonstrate masking functions
-- NO ABAC POLICIES REQUIRED - Manual function demonstration
-- =============================================

## Test: TABLE ROW COUNTS


In [0]:
-- =============================================
-- TEST 1: Table Row Counts
-- =============================================

SELECT 
  'customers' AS table_name, 
  COUNT(*) AS row_count 
FROM customers
UNION ALL SELECT 'accounts', COUNT(*) FROM accounts
UNION ALL SELECT 'credit_cards', COUNT(*) FROM credit_cards
UNION ALL SELECT 'transactions', COUNT(*) FROM transactions
UNION ALL SELECT 'loans', COUNT(*) FROM loans
ORDER BY table_name;

## Test: CREDIT CARD MASKING (PCI-DSS)


In [0]:
-- =============================================
-- TEST 2: Credit Card Masking Demo
-- =============================================

SELECT 
  card_id,
  customer_id,
  card_number AS original,
  mask_credit_card(card_number) AS masked,
  card_type
FROM credit_cards
LIMIT 5;

## Test: ACCOUNT NUMBER MASKING


In [0]:
-- =============================================
-- TEST 3: Account Number Masking Demo
-- =============================================

SELECT 
  account_id,
  customer_id,
  account_number AS original,
  mask_account_last4(account_number) AS masked_last4,
  account_type
FROM accounts
LIMIT 5;

## Test: SSN MASKING


In [0]:
-- =============================================
-- TEST 4: SSN Masking Demo
-- =============================================

SELECT 
  customer_id,
  CONCAT(first_name, ' ', last_name) AS name,
  ssn AS original,
  mask_ssn_last4(ssn) AS masked,
  credit_score
FROM customers
LIMIT 5;

## Test: EMAIL & PHONE MASKING


In [0]:
-- =============================================
-- TEST 5: Email & Phone Masking Demo
-- =============================================

SELECT 
  customer_id,
  first_name,
  email AS original_email,
  mask_email(email) AS masked_email,
  phone AS original_phone,
  mask_phone(phone) AS masked_phone
FROM customers
LIMIT 5;

## Test: AMOUNT BUCKETING


In [0]:
-- =============================================
-- TEST 6: Transaction Amount Bucketing
-- =============================================

SELECT 
  transaction_id,
  amount AS original_amount,
  mask_amount_bucket(amount) AS amount_bucket,
  merchant_name,
  transaction_type
FROM transactions
ORDER BY amount DESC
LIMIT 10;

## Test: ROUTING NUMBER MASKING


In [0]:
-- =============================================
-- TEST 7: Routing Number Masking
-- =============================================

SELECT 
  account_id,
  routing_number AS original,
  mask_routing_number(routing_number) AS masked,
  account_type
FROM accounts
WHERE routing_number IS NOT NULL
LIMIT 5;

## Test: TRANSACTION ID HASHING


In [0]:
-- =============================================
-- TEST 8: Transaction ID Hashing
-- =============================================

SELECT 
  transaction_id AS original,
  mask_transaction_hash(transaction_id) AS hashed,
  amount,
  merchant_name
FROM transactions
LIMIT 5;

## Test: CUSTOMER ID MASKING (Deterministic)


In [0]:
-- =============================================
-- TEST 9: Customer ID Deterministic Masking
-- =============================================

SELECT 
  customer_id AS original,
  mask_customer_id_deterministic(customer_id) AS masked,
  first_name,
  last_name
FROM customers
LIMIT 5;

## Test: IP ADDRESS MASKING


In [0]:
-- =============================================
-- TEST 10: IP Address Masking
-- =============================================

SELECT 
  transaction_id,
  ip_address AS original,
  mask_ip_address(ip_address) AS masked,
  device_type
FROM transactions
LIMIT 5;

## Test: INCOME BRACKETING


In [0]:
-- =============================================
-- TEST 11: Income Bracket Masking
-- =============================================

SELECT 
  customer_id,
  CONCAT(first_name, ' ', last_name) AS name,
  annual_income AS original,
  mask_income_bracket(annual_income) AS income_bracket
FROM customers
ORDER BY annual_income DESC
LIMIT 5;

## Test: MULTI-COLUMN MASKING


In [0]:
-- =============================================
-- TEST 12: Multi-Column Masking Demo
-- =============================================

SELECT 
  customer_id,
  mask_customer_id_deterministic(customer_id) AS masked_id,
  CONCAT(first_name, ' ', last_name) AS name,
  mask_ssn_last4(ssn) AS masked_ssn,
  mask_email(email) AS masked_email,
  mask_phone(phone) AS masked_phone,
  mask_income_bracket(annual_income) AS income_bracket
FROM customers
LIMIT 5;

## Test: FRAUD FLAGGED TRANSACTIONS


In [0]:
-- =============================================
-- TEST 13: Fraud Transaction Detection
-- =============================================

SELECT 
  transaction_id,
  customer_id,
  amount,
  merchant_name,
  fraud_flag,
  filter_fraud_flagged_only(fraud_flag) as fraud_filter,
  status
FROM transactions;

## Test: HIGH-VALUE TRANSACTIONS (>$5000)


In [0]:
-- =============================================
-- TEST 14: High-Value Transactions
-- =============================================

SELECT 
  transaction_id,
  customer_id,
  amount,
  filter_high_value_transactions(amount) AS amount_filter,
  merchant_name,
  transaction_date
FROM transactions
ORDER BY amount DESC;

## Test: BUSINESS HOURS FILTER

In [0]:
-- =============================================
-- TEST 15: Business Hours Filter
-- =============================================

SELECT 
  transaction_id,
  customer_id,
  hour(current_timestamp()),
  filter_business_hours() AS hours_filter,
  merchant_name
FROM transactions;

## Test: REGIONAL ACCESS FILTER

In [0]:
-- =============================================
-- TEST 16: Region Access Filter
-- =============================================

SELECT 
  customer_id,
  email,
  state,
  filter_by_region(state) AS region_filter,
  credit_score
FROM customers;

## ✅ All Tests Complete!

Congratulations! All Finance ABAC functions are working correctly!

### What You Verified:
- ✅ All tables contain expected data
- ✅ Masking functions produce correct output
- ✅ Data transformations maintain privacy requirements
- ✅ Functions are ready for ABAC policy integration

### Test Summary:
- **Email Masking**: ✅ Local part hidden, domain visible
- **Phone Masking**: ✅ Showing last 4 digits only
- **Financial Data**: ✅ Bucketed or last-4 protected
- **Identifiers**: ✅ Deterministic hashing working
- **Sensitive Fields**: ✅ Complete redaction successful

### 🎯 Next Steps - Implementing ABAC Policies:

Now that masking functions are tested, you can:

1. **Create User Groups**:
   ```sql
   -- Example: Create groups for different access levels
   CREATE GROUP IF NOT EXISTS finance_analysts;
   CREATE GROUP IF NOT EXISTS finance_admins;
   ```

2. **Apply Column Tags**:
   ```sql
   -- Example: Tag sensitive columns
   ALTER TABLE your_catalog_name.finance.<table_name>
   ALTER COLUMN <column_name> SET TAGS ('PII' = 'email');
   ```

3. **Create ABAC Policies**:
   ```sql
   -- Example: Apply masking based on tags
   CREATE OR REPLACE POLICY your_catalog_name.finance.apply_pii_policy ON CATALOG your_catalog_name
   COLUMN MASK mask_email
   TO finance_analysts
   EXCEPT finance_admins
   FOR TABLES MATCH COLUMNS hasTagValue('PII', 'email') AS email
   ON COLUMN email;
   ```

4. **Test Policies**:
   - Log in as different users
   - Query the same table
   - Verify each user sees appropriately masked data

### 📚 Additional Resources:
- [Unity Catalog ABAC Documentation](https://docs.databricks.com/aws/en/data-governance/unity-catalog/abac/)
- [Row Filters and Column Masks](https://docs.databricks.com/aws/en/data-governance/unity-catalog/filters-and-masks)

---
**🎉 Great Job!** Your Finance ABAC demo foundation is complete and tested!
